# Exercise 12: Test the Dropout Layer
## SIB - Intelligent Systems for Bioinformatics

Bárbara Freitas PG55693

In [2]:
import sys
import os
import numpy as np
from unittest import TestCase

# Adiciona a pasta raiz ao path para garantir que os imports funcionam
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..')))

from si.io.csv_file import read_csv
from si.models.logistic_regression import LogisticRegression
from si.model_selection.randomized_search import randomized_search_cv
from si.metrics.accuracy import accuracy

class TestRandomizedSearchCV(TestCase):

    def setUp(self):
        # Configurar o caminho para o dataset breast-bin
        datasets_base_path = os.path.join(os.path.dirname(__file__), '..', '..', 'datasets')
        self.filename = os.path.join(datasets_base_path, 'breast_bin', 'breast-bin.csv')
        self.dataset = read_csv(self.filename, sep=",", features=True, label=True)

        # Configurar o modelo
        self.model = LogisticRegression()

        # Configurar a grelha de hiperparâmetros (exatamente como no exercício)
        self.hyperparameter_grid = {
            'l2_penalty': np.linspace(1, 10, 10),
            'alpha': np.linspace(0.001, 0.0001, 100),
            'max_iter': np.linspace(1000, 2000, 200, dtype=int)
        }

    def test_randomized_search_execution(self):
        """
        Testa a execução do randomized_search_cv com o dataset breast-bin.
        """
        n_iter = 10
        cv_folds = 3

        # Executar a função
        results = randomized_search_cv(
            model=self.model,
            dataset=self.dataset,
            hyperparameter_grid=self.hyperparameter_grid,
            scoring=accuracy,
            cv=cv_folds,
            n_iter=n_iter,
            random_state=42
        )

        # 1. Verificar se o output é um dicionário e tem as chaves corretas
        self.assertIsInstance(results, dict)
        expected_keys = ['hyperparameters', 'scores', 'best_hyperparameters', 'best_score']
        for key in expected_keys:
            self.assertIn(key, results)

        # 2. Verificar se o número de resultados corresponde ao n_iter
        self.assertEqual(len(results['scores']), n_iter)
        self.assertEqual(len(results['hyperparameters']), n_iter)

        # 3. Verificar se o melhor score é válido (entre 0 e 1 para accuracy)
        # O breast-bin costuma dar ~0.96, por isso deve ser maior que 0.90
        self.assertTrue(0.0 <= results['best_score'] <= 1.0)
        self.assertGreater(results['best_score'], 0.90)

        # 4. Verificar se os melhores hiperparâmetros foram escolhidos da grelha
        best_params = results['best_hyperparameters']
        self.assertIn('l2_penalty', best_params)
        self.assertIn('alpha', best_params)
        self.assertIn('max_iter', best_params)

        # Verificar se os valores escolhidos pertencem às distribuições originais
        # Nota: Usamos verificação aproximada para floats
        l2_check = any(np.isclose(best_params['l2_penalty'], x) for x in self.hyperparameter_grid['l2_penalty'])
        self.assertTrue(l2_check, "O l2_penalty escolhido não está na grelha original")

NameError: name '__file__' is not defined